In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np
import datetime as dt
from datetime import datetime

from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [5]:
url = "https://www.aruodas.lt/namai/vilniuje/antakalnyje/"
response = req.get(url)

if(response.status_code == 200):
    print("Page is accessible")
else:
    print("Page is not accessible, error code: {}".format(response.status_code))

Page is not accessible, error code: 403


In [6]:
flatsPriceData = pd.read_excel('../DistrictPriceAnalysis/FlatsPriceDataPivot.xlsx')

In [7]:
xls = pd.ExcelFile('../DistrictPriceAnalysis/FlatsPriceDataPivot.xlsx')
flatsPriceData = pd.read_excel(xls, 'SortedData')
flatsPriceData.head()

,City,District,Sale_m2,Rent_m2,RelativePriceToRent
0,Kaunas,Sargėnai,978.00,7.65,127.83
1,Vilnius,Naujoji Vilnia,1216.42,8.99,135.27
2,Kaunas,Aukštieji Šančiai,1280.45,8.86,144.57
3,Klaipėda,Žvejybos uostas,1096.83,7.07,155.05
4,Panevėžys,Centras,1030.73,6.56,157.08


In [4]:
#flatsPriceData.iloc[0][1]

In [8]:
def GetDistrictUrl(city, district):
    url = "https://en.aruodas.lt/"
    driver = webdriver.Chrome(r"C:/Users/Paulius/Downloads/chromedriver_win32 (1)/chromedriver.exe")
    driver.get(url)
    sleep(1)
    driver.find_element_by_id('onetrust-accept-btn-handler').click()
    sleep(1)   
    driver.find_element(By.ID, 'display_FRegion').click()
    sleep(0.5)
    cityInput = driver.find_element(By.ID, "filterInput_FRegion")
    cityInput.send_keys(city)
    sleep(0.5)
    cityInput.send_keys(Keys.ENTER)
    sleep(0.5)
    driver.find_element(By.ID, 'display_FQuartal').click()
    sleep(2)
    driver.switch_to.frame(driver.find_element(By.ID, "fancybox-frame"))
    sleep(1)
    driver.find_element_by_xpath("//label[@title='{}']".format(district)).click()
    sleep(0.5)
    driver.find_element(By.ID, 'btSaveSelected').click()
    sleep(0.5)
    driver.switch_to.default_content()
    sleep(0.5)
    driver.find_element(By.ID, 'buttonSearchForm').click()
    currentUrl = driver.current_url
    driver.close()
    return currentUrl

In [89]:
GetDistrictUrl(flatsPriceData.iloc[0][0],flatsPriceData.iloc[0][1])

'https://en.aruodas.lt/butai/kaune/sargenuose/'

In [9]:
def GetDistrictPropertyData(districtUrl, city, district, globalDataFrame):
    driver = webdriver.Chrome(r"C:/Users/Paulius/Downloads/chromedriver_win32 (1)/chromedriver.exe")
    driver.get(districtUrl)
    sleep(1)
    driver.find_element_by_id('onetrust-accept-btn-handler').click()
    sleep(1)
    
    soup = bs(driver.page_source, "lxml")
    pagination = soup.find("div", {"class" : "pagination"})
    maxPages = 1
    if(pagination is not None and len(pagination) > 0):
        maxPages = (len(pagination.find_all("a", {"class" : "page-bt"})))
    
    for page in range(1,maxPages+1):
        url = districtUrl + "puslapis/" + str(page) + "/"       
        driver.get(url)
        sleep(1)
        soup = bs(driver.page_source, "lxml")
        allElements = soup.find_all("tr", {"class" : "list-row"})
        allListings = [element.find('h3').find('a', href=True)['href'] for element in allElements if element.find("td", {"class":"list-adress"}) != None]
        
        for propertyUrl in allListings:
            driver.get(propertyUrl)
            sleep(1)
            soup = bs(driver.page_source, "lxml")
            
            price = soup.find("span", {"class" : "price-eur"}).text.split("€")[0].strip().replace(" ", "")
            pricePerM2 = soup.find("span", {"class" : "price-per"}).text.split("€")[0].strip().split("(")[1].replace(" ", "")         
            
            mainTable = soup.find("dl", {"class" : "obj-details"})
            area = mainTable.find('dt', text=re.compile('Area')).find_next_sibling('dd').text.strip().split(" ")[0].replace(",", ".")    
            floor = mainTable.find('dt', text=re.compile('Floor')).find_next_sibling('dd').text.strip()
            numberOfFloors = mainTable.find('dt', text=re.compile('No. of floors')).find_next_sibling('dd').text.strip()
            roomCount = mainTable.find('dt', text=re.compile('Number of rooms')).find_next_sibling('dd').text.strip()
            builtYear = mainTable.find('dt', text=re.compile('Build year')).find_next_sibling('dd').text.strip()
            heating = mainTable.find('dt', text=re.compile('Heating system')).find_next_sibling('dd').text.strip()
            equipment = mainTable.find('dt', text=re.compile('Equipment')).find_next_sibling('dd').text.strip()
            dataTable = pd.DataFrame({'Price': [price], 'PricePerM2': [pricePerM2], 'Area': [area], 'floor' : [floor], 'Number of floors': [numberOfFloors], 'Room count': [roomCount], 
                                      'Built year': [builtYear], 'Heating': [heating], 'Equipment': [equipment], 'City': [city], 'District': [district], 'href':[propertyUrl]})
            globalDataFrame = pd.concat( [globalDataFrame, dataTable])
            #print(globalDataFrame)
            driver.back()
            sleep(1)        
    driver.close()
    return globalDataFrame

In [37]:
globalDataFrame = pd.DataFrame()
table = GetDistrictPropertyData("https://en.aruodas.lt/butai/vilniuje/seskineje/",flatsPriceData.iloc[0][0],flatsPriceData.iloc[0][1], globalDataFrame)

In [10]:
def GetAllListings(flatsPriceData):
    global globalDataFrame
    for index in range(0,85):
        districtUrl = GetDistrictUrl(flatsPriceData.iloc[index][0], flatsPriceData.iloc[index][1])
        globalDataFrame = GetDistrictPropertyData(districtUrl, flatsPriceData.iloc[index][0], flatsPriceData.iloc[index][1], globalDataFrame)
        print(index)
    return globalDataFrame

In [3]:
globalDataFrame = pd.DataFrame()
allListings = GetAllListings(flatsPriceData)

In [203]:
def ChangeDataTypesAndCleanData(df):
    df = df.reset_index()
    df["Price"] = pd.to_numeric(df["Price"])
    df["PricePerM2"] = pd.to_numeric(df["PricePerM2"])
    df["Area"] = pd.to_numeric(df["Area"])
    df["floor"] = pd.to_numeric(df["floor"])
    df["Number of floors"] = pd.to_numeric(df["Number of floors"])
    df["Room count"] = pd.to_numeric(df["Room count"])
    df['Built year'] = df['Built year'].apply(lambda x: x[0:4])
    df["Built year"] = pd.to_datetime(df['Built year'].apply(lambda x: x[0:4]) + "-01-01")
    
    df["Equipment"] = df["Equipment"].apply(lambda x: "Partial decoration" if "USEFUL" in x else x)
    return df

In [1]:
allListings = ChangeDataTypesAndCleanData(allListings)

In [205]:
allListings = allListings.reset_index(drop=True)

In [206]:
df = pd.DataFrame()
df['date2'] = pd.to_datetime(allListings['Built year'].apply(lambda x: x[0:4]) + "-01-01", errors = 'coerce')
df = df.reset_index(drop=True)

In [207]:
indexesForCleaning = df.loc[df['date2'].isna()].index.array
indexesForCleaning

<PandasArray>
[712, 1738, 2137, 2746, 4044, 4117]
Length: 6, dtype: int64

In [208]:
allListings['Built year'].iloc[712]
#allListings['href'].iloc[712]

'1000'

In [209]:
allListings.iloc[712]

Price                                                           36000
PricePerM2                                                       1500
Area                                                               24
floor                                                               3
Number of floors                                                    5
                                          ...                        
Heating                                                       Central
Equipment                                              Fully equipped
City                                                           Kaunas
District                                                      Dainava
href                https://en.aruodas.lt/butai-kaune-dainavoje-v-...
Name: 712, Length: 12, dtype: object

In [210]:
for index in indexesForCleaning:
    indexesForCleaning = df.loc[df['date2'].isna()].index.array
    cityForCleaning = allListings['City'].iloc[index]
    districtForCleaning = allListings['District'].iloc[index]
    withoutProblematicDates = allListings[~allListings.index.isin(indexesForCleaning)]
    filter1 = withoutProblematicDates['City'] == cityForCleaning
    filter2 = withoutProblematicDates['District'] == districtForCleaning

    withoutProblematicDates = withoutProblematicDates.loc[filter1 * filter2]
    dates = pd.to_datetime(withoutProblematicDates['Built year'].apply(lambda x: x[0:4]) + "-01-01")
    medianDate = int(np.median(np.array(dates.map(dt.datetime.toordinal))))
    medianDate = dt.datetime.fromordinal(medianDate)
    year = str(medianDate)[0:4]
    allListings["Built year"].iloc[index] = year

In [212]:
print(allListings['Built year'].iloc[712])
print(allListings['Built year'].iloc[1738])

1973
1984


In [213]:
allListings = ChangeDataTypesAndCleanData(allListings)

In [214]:
allListings.to_csv('./AllListings.csv', index=False, encoding='utf-8-sig')

In [249]:
pd.set_option('display.max_rows', 10)
allListingsCount = (allListings.groupby(['City', 'District']).size().reset_index(name='ListingsCount'))
eligibleListings = allListingsCount[allListingsCount["ListingsCount"]>=20].reset_index(drop=True)
eligibleListings.to_csv('./EligibleListings.csv', index=False, encoding='utf-8-sig')

In [270]:
allEligiblePropertiesFilter = allListings["District"].apply(lambda x: True if x in eligibleListings["District"].array else False)
allEligibleListings = allListings[allEligiblePropertiesFilter].drop("index", axis = 1)
allEligibleListings.to_csv('./AllEligibleListings.csv', index=False, encoding='utf-8-sig')